![EncabezadoIN](EncabezadoIN.JPG)

## Laboratorio 2 - Regresión

Integrantes: 
Juan Pablo Lora Hernández - 202012524 

Andrés Francisco Borda Rincón - 201729184

Gabriela Vargas Rojas - 202013830


## Caso 
CompuAlpes es una reconocida tienda minorista que vende computadores portátiles de diferentes fabricantes y especificaciones técnicas. Con el auge de la tecnología y el creciente número de productos en el mercado, la empresa busca optimizar sus estrategias de fijación de precios y promociones para seguir siendo competitiva. Es en este último punto, donde ha identificado un reto relacionado con determinar el precio adecuado para un portatil ya que el mercado es dinámico y la valoración de las características técnicas cambia con el tiempo. Poner un precio demasiado alto puede alejar a los clientes, mientras que ponerlo demasiado bajo puede reducir los márgenes de ganancia.
Esto motivó a CompuAlpes a proponer el objetivo de este proyecto, en el cual se desea construir un modelo de regresión que permita estimar el precio de un portátil a partir de sus especificaciones técnicas, determinando las que más impactan en el precio o que son, de acuerdo a la evidencia, irrelevantes para la estimación. Este modelo permitirá a CompuAlpes tener una base objetiva y cuantitativa al momento de establecer precios para sus productos.

## 1. Carga de librerías necesarias para implementación

In [2]:
import numpy as np
import pandas as pd

from joblib import dump, load

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import scipy.stats as stats

## 2. Cargar los datos

In [3]:
# Se cargan los datos de entrenamiento.
df_compu=pd.read_csv('./Regresión_data_laptop_data_train.csv', sep=';', encoding = "ISO-8859-1")

## 3. Entendimiento de los datos 

In [4]:
df_compu.shape

(1216, 1)

In [7]:
df_compu.dtypes

id,Company,TypeName,Ram,Weight,Price,TouchScreen,Ips,Ppi,Cpu_brand,HDD,SSD,Gpu_brand,Os    object
dtype: object

In [8]:
df_compu.sample(5)

,"id,Company,TypeName,Ram,Weight,Price,TouchScreen,Ips,Ppi,Cpu_brand,HDD,SSD,Gpu_brand,Os"
1185,"970,Dell,2 in 1 Convertible,16,1.22,11.5246437..."
1134,"488,Lenovo,Ultrabook,16,1.36,11.4699912412637,..."
509,"938,Dell,Ultrabook,8,1.36,11.377402604975964,0..."
835,"599,Dell,Workstation,8,2.06,11.450333208630004..."
399,"1144,HP,2 in 1 Convertible,8,1.48,11.470547260..."


En este punto, algo propio de la regresión es determinar las variables candidatas a incluir, por lo cual se sugiere entender las relaciones entre las variables de entrada y la variable objetivo.